In [1]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.models import Model, Sequential
from keras.layers import Convolution1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, LSTM, GRU, CuDNNGRU, CuDNNLSTM, concatenate, Input, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.constraints import maxnorm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Input -> embed_size,maxlen,max_features,lr
#Output -> model()

#Multilayer - CNN - RNN model
def get_model_1(embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    x = Convolution1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Convolution1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = Convolution1D(128*2**(strides), strides = strides, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_size=3, kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x = concatenate([x_f, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['binary_accuracy'])
    return model

#MLP Model
def get_model_2 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dense(128, activation="relu")(x)    
    x = Flatten()(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#CNN Model
def get_model_3 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Convolution1D(2 * embed_size, kernel_size = 2, activation = "relu")(x)    
    x = Flatten()(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#2 layers with Dropout - CNN Model
def get_model_4 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Convolution1D(2 * embed_size, kernel_size = 2, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#3 layers with Dropout - CNN Model
def get_model_5 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Convolution1D(2 * embed_size, kernel_size = 2, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Convolution1D(embed_size, kernel_size = 3, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#Simple RNN with Dropout
def get_model_6 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = SimpleRNN(256) (x)
    x = Dropout(0.10)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#Simple LSTM with Dropout
def get_model_7 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = CuDNNLSTM(256) (x)
    x = Dropout(0.10)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#Simple GRU with Dropout
def get_model_8 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = CuDNNGRU(256) (x)
    x = Dropout(0.10)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#CNN-RNN model with Dropout
def get_model_9 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Convolution1D(2 * embed_size, kernel_size = 2, activation = "relu")(x)   
    x = Dropout(0.10)(x)
    x = CuDNNLSTM(256) (x)
    x = Dropout(0.10)(x)    
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#RNN-CNN model with Dropout
def get_model_10 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = CuDNNLSTM(256, return_sequences=True) (x)
    x = Dropout(0.10)(x)   
    x = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)   
    x = Dropout(0.25)(x)
    x = Convolution1D(2 * embed_size, kernel_size = 2, activation = "relu")(x)   
    x = Dropout(0.10)(x)
    x = Flatten()(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#CNN-RNN model with Concatenate and Dropout
def get_model_11 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    #Branch A
    x_a = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)
    x_a = Dropout(0.10)(x_a)
    x_a = CuDNNLSTM(512)(x_a)  
    #Branch B
    x_b = Convolution1D(embed_size, kernel_size = 2, activation = "relu")(x)
    x_b = Dropout(0.10)(x_b)
    x_b = CuDNNLSTM(256, return_sequences=True)(x_b)  
    x_b = Dropout(0.10)(x_b)
    x_b = CuDNNLSTM(128)(x_b)  
    #Concatenate Branch A-B
    x = concatenate([x_a, x_b])
    x = Dropout(0.25)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model

#CNN-RNN-MLP model with Concatenate and Dropout
def get_model_12 (embed_size,maxlen,max_features,lr):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    #Branch A - 2 layers CNN + MLP
    x_a = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)
    x_a = Dropout(0.10)(x_a)
    x_a = Convolution1D(2 * embed_size, kernel_size = 3, activation = "relu")(x)
    x_a = Dropout(0.10)(x_a)
    x_a = Dense(512, activation="relu")(x_a)
    x_a = Flatten()(x_a)
    #Branch B - RNN + MLP
    x_b = Convolution1D(embed_size, kernel_size = 2, activation = "relu")(x)
    x_b = Dropout(0.10)(x_b)
    x_b = CuDNNLSTM(256, return_sequences=True)(x_b)  
    x_b = Dropout(0.10)(x_b)
    x_b = Dense(512, activation="relu")(x_b)
    x_b = Flatten()(x_b)
    #Concatenate Branch A-B
    x = concatenate([x_a, x_b])
    x = Dropout(0.25)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])
    return model